## Spain Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Investing.com MAIN: https://www.investing.com /
Investing.com Spain: https://www.investing.com/equities/spain

Description: This is public company financial statement web crawling code for investing.com, especially for companies in Spain. investing.com is an unofficial stock information website for countries around the world. Even though it is an unofficial website, comparably, it has decent amount of information including key financial values. However, the server itself is very unstable that loading error occurs a lot, and data table format/language format is not standardized that a lot of exceptions are made(goes to failList). So one should be aware of that, and it is better to find alternative source for a long term development. Using excel format instead of csv could be a solution, which I have not tried so far. This code can be reused for other countries by editing initial website address and several page setting button XPATH locations - Colombia, Indonesia, Italy, Sinbgapore, and Spain is already written and uploaded. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [4]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment### 

In [5]:
cover = open('Spain_2022_investing_2022-07-04_Chris.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - One also need to avoid ad and accept cookie banner that randomly appears.
#### - Run the crawler many times to improve the accuracy - Detail is written below main crawler.

In [6]:
# Basic Setting
failList = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/spain')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(4)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(1)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가정보
        info.append("ESP") 
        info.append("Espana") 
        info.append("Spain") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2200000000000 USD") 
        info.append("47450795") 
        info.append("서유럽") 

        #기업코드
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ESPHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Spain, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain, Europe"
                post = "N/A"
            except:
                address1 = "Spain"
                address2 = "Spain, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Espanol")
        info.append("Spanish")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BME")
        info.append("Bolsas de Valores")
        info.append("Bolsas de Valores")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Spain")
        info.append("Spain, Europe")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_7632\2364196304.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
<table class="genTbl closedTbl crossRatesTbl elpTbl elp25" id="cross_rate_markets_stocks_1" tablesorter=""><thead><tr><th class="flag"> </th><th class="left noWrap elp pointer">Name<span class="headerSortDefault newSiteIconsSprite" sort_default=""></span></th><th>Last</th><th>High</th><th>Low</th><th data-col-caption="Change">Chg.</th><th class="pointer" data-col-caption="Change %">Chg. %<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Vol">Vol.<span class="headerSortUp newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Time">Time<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="icon"></th></tr></thead><tbody><tr id="pair_32228"><td class="flag"><span class="ceFlags Spain" title="Spain"> </span></td><td class="bold left noWrap elp plusIconTd"><a href="/equities/nyesa-valores-corp" title="Ny

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0150480111', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA is a Spain-based company primarily engaged in the real estate sector. The Company’s activities include the acquisition and development of land, as well as the promotion and construction of non-residential properties, such as hotels, office buildings, shopping centers, senior residences, as well as logistics and industrial properties. The Company owns such companies as Gestora Inmobiliaria Besos SA, Edutaimet Sant Adria de Besos SA, Promociones Industriales y Financieras SA, Constructora Inbesos SA, Inbesos Sur SA and Nalcar 2000 SL, among others. On December 5, 2013, Commercial Court No. 1 of Zaragoza completed the common phase of insolvency proceedings. In December 2013, the trustee in bankruptcy presented final report of the Company required under article 96 of t

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0140609019', 'Caixabank SA (CABK)', 'Caixabank SA (CABK)', 'CaixaBank, S.A., together with its subsidiaries, provides various banking products and financial services in Spain and internationally. The company operates through Banking and Insurance, Equity Investments, and BPI segments. It also provides solutions related to security, protection, internationalization, and financing; traditional financial advice, independent advice, and broker services; asset management; liquidity management; capital markets, cash management, project finance, asset finance, and M&A services; and various financial services and solutions to public and private sector institutions, as well as distributes non-life and life risk insurance policies; and private banking services. In addition, the company is involved in the real estate business. As of September 30, 2021, it had 5,415 branches and 13,678 ATMs in S

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0124244E34', 'Mapfre (MAP)', 'Mapfre (MAP)', 'Mapfre, S.A., engages in the insurance and reinsurance activities worldwide. It offers life, health, accident, savings and investment, retirement, burial, and travel and leisure insurance; and homeowner’s, automobile, third-party liability, family, and other insurance. The company also provides vehicle, third-party liability and asset, agriculture and livestock, commercial establishment, and other insurance products. In addition, it offers engineering and building, hull and aviation, transportation of goods, surety and credit, life and retirement, and other insurances and reinsurance products. The company offers its services to individuals, professionals, entrepreneurs, self-employed people, small and medium-sized enterprises, and large corporations. It distributes its products through a network of 4,942 direct and delegates; and 10,412 b

...

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0126251018', 'Desarrollos Especiales de Sistemas de Anclajes SA (DESA)', 'Desarrollos Especiales de Sistemas de Anclajes SA (DESA)', 'Desarrollos Especiales de Sistemas de Anclaje, S.A. manufactures and sells anchors and fastenings primarily in Spain. It offers metallic, chemical, and plastic anchors; screws, and threaded sockets and spouts; rivets, and shank and threaded rivets; power drive, gad drive, tips, sockets, and spikes; and chemical solutions, such as foam and silicon applicator guns, putties, MS polymer, and adhesives. The company also provides installation systems, which include rails and accessories, and radiator and WC supports; clamps, nylon cable ties, and perforated tapes; and tying system in trusses. Its products are used for construction and industrial supplies, building materials, and plates, as well as air conditioning, plumbing, and electricity warehouses.It off

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105568002', 'Media Investment Optimization SL (MIO)', 'Media Investment Optimization SL (MIO)', 'Media Investment Optimization SA engages in the marketing and advertising business in Spain. It offers radio marketing and advertising, marketing strategy and media management, business consulting, branded content, and media advertising consulting services. The company is also involved in media buying, data analytics, web development, marketing automation, and technology. Media Investment Optimization SA was founded in 2012 and is based in Madrid, Spain.', 'Media Investment Optimization SL (MIO)(English: Media Investment Optimization SL (MIO)) is a public company that is listed on Spain Stock Exchange.', 'Media Investment Optimization SL (MIO)(English: Media Investment Optimization SL (MIO)) is a public company that is listed on Spain Stock Exchange.', None, None, '', '', '', '34 902 333

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105591004', 'Llorente & Cuenca Madrid SL (LLYC)', 'Llorente & Cuenca Madrid SL (LLYC)', 'Llorente & Cuenca, S.A. operates as a communications and public affairs consulting company in Latin America, Spain, Portugal, and internationally. The company offers services in the areas of public affairs, consumer engagement, corporate operations and capital markets, economic context, digital, stakeholders management, talent engagement, creative studio, crisis and risks, sport and business strategy, digital transformation, litigation, inbound marketing, data and analytics, CSR and corporate foundations, branding, investor relations, and executive coaching. It serves various industries, such as public administrations and multilateral organizations; information and communication technologies; education; mining; food and beverage; finance; energy, oil, and gas; health and pharmaceuticals; tourism

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105436002', 'Greenoak Spain Holdings Socimi II SA (YGO2)', 'Greenoak Spain Holdings Socimi II SA (YGO2)', 'Greenoak Spain Holdings Socimi II S.A. engages in the acquisition, development, leasing, and management of real estate assets in Spain. The company was founded in 2016 and is based in Madrid, Spain.', 'Greenoak Spain Holdings Socimi II SA (YGO2)(English: Greenoak Spain Holdings Socimi II SA (YGO2)) is a public company that is listed on Spain Stock Exchange.', 'Greenoak Spain Holdings Socimi II SA (YGO2)(English: Greenoak Spain Holdings Socimi II SA (YGO2)) is a public company that is listed on Spain Stock Exchange.', None, None, '', '', '', '34 91 414 86 67', '-', '', '', '', '', '', '', '', 'Calle Pinar 7 5 Izquierda, Spain', 'Calle Pinar 7 5 Izquierda, Spain', '28006', '28006', '28006', '-', '', '', 'Listed', '', '', 'Director', 'Director', 'Executive Board', '34 91 414 86 67

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105030003', 'Mercal Inmuebles SOCIMI SA (YMEI)', 'Mercal Inmuebles SOCIMI SA (YMEI)', 'Mercal Inmuebles Socimi, S.A. engages in the leasing and managing real estate properties. Its portfolio consists of offices, hospitals, land, and garages. The company is based in Madrid, Spain.', 'Mercal Inmuebles SOCIMI SA (YMEI)(English: Mercal Inmuebles SOCIMI SA (YMEI)) is a public company that is listed on Spain Stock Exchange.', 'Mercal Inmuebles SOCIMI SA (YMEI)(English: Mercal Inmuebles SOCIMI SA (YMEI)) is a public company that is listed on Spain Stock Exchange.', None, None, '', '', '', '630 17 95 63', '-', '', 'www.mercalinmuebles.es', '', '', '', '', '', 'Calle Orense 81 7th Floor, Spain', 'Calle Orense 81 7th Floor, Spain', '28020', '28020', '28020', '1', '', '', 'Listed', 'José Enrique García Ramos', '', 'CEO, Secretary & Director', 'CEO, Secretary & Director', 'Executive Board', '63

### 4. Dealing with FailList
#### - Because Investing.com causes loading time error due to unstable server status, it is better to run the code, at least with companies in failList, for several times or run with two or more computers. That will increase the sucess rate much higher than running the code once. When running a new code, make sure to change the name of saving file so that you don't replace the original file.
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).
#### - Below is both actual and sample code that was used in 2022

In [7]:
print(len(failList))

160


In [8]:
cover = open('Spain_2022_investing_2022-07-04_Chris-2.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [9]:
# Basic Setting
failList2 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/spain')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(2)
    
for sub in failList:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ESP") 
        info.append("Espana") 
        info.append("Spain") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2200000000000 USD") 
        info.append("47450795") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ESPHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Spain, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain, Europe"
                post = "N/A"
            except:
                address1 = "Spain"
                address2 = "Spain, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Espanol")
        info.append("Spanish")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BME")
        info.append("Bolsas de Valores")
        info.append("Bolsas de Valores")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Spain")
        info.append("Spain, Europe")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList2.append(sub)
            
print(failList2)

C:\Users\user\AppData\Local\Temp\ipykernel_7632\2214047812.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0150480111', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA is a Spain-based company primarily engaged in the real estate sector. The Company’s activities include the acquisition and development of land, as well as the promotion and construction of non-residential properties, such as hotels, office buildings, shopping centers, senior residences, as well as logistics and industrial properties. The Company owns such companies as Gestora Inmobiliaria Besos SA, Edutaimet Sant Adria de Besos SA, Promociones Industriales y Financieras SA, Constructora Inbesos SA, Inbesos Sur SA and Nalcar 2000 SL, among others. On December 5, 2013, Commercial Court No. 1 of Zaragoza completed the common phase of insolvency proceedings. In December 2013, the trustee in bankruptcy presented final report of the Company required under artic

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0124244E34', 'Mapfre (MAP)', 'Mapfre (MAP)', 'Mapfre, S.A., engages in the insurance and reinsurance activities worldwide. It offers life, health, accident, savings and investment, retirement, burial, and travel and leisure insurance; and homeowner’s, automobile, third-party liability, family, and other insurance. The company also provides vehicle, third-party liability and asset, agriculture and livestock, commercial establishment, and other insurance products. In addition, it offers engineering and building, hull and aviation, transportation of goods, surety and credit, life and retirement, and other insurances and reinsurance products. The company offers its services to individuals, professionals, entrepreneurs, self-employed people, small and medium-sized enterprises, and large corporations. It distributes its products through a network of 4,942 direct and delegates; and 10,412 b

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0180907000', 'Unicaja Banco SA (UNI)', 'Unicaja Banco SA (UNI)', 'Unicaja Banco, S.A. provides various banking products and services to individuals and companies in Spain and internationally. It offers accounts, payments, and debit and credit cards; mortgages and personal loans; deposit products, stock exchange, pension plans, investment funds and portfolios, and savings insurance policies; and life, home, car, accident, health, and agricultural insurance, as well as SME and retail damage, and corporate liability insurance products. The company also provides cash management, short term and long-term financing, and investment services, as well as remote and mobile banking services. In addition, it engages in the property development and renewable energies activities; and invests in assets, securities, and financial companies. As of December 31, 2021, it had a network of 1,368 branches

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0132105018', 'Acerinox (ACX)', 'Acerinox (ACX)', 'Acerinox, S.A., through its subsidiaries, manufactures, transforms, and markets stainless steel products in Spain, the Americas, Africa, Asia, Oceania, and Europe. The company offers flat products, including coil cold rollings, hot rolled and black coils, teardrop steel or coils, and hot and cold rolled sheets, as well as roughing materials, discs, billets, and plates. It also provides long products, which include steel and color coated wires, corrugated wires, hexagonal wire rods, bars, hot and cold rebars, decorticated bars, black bars, steel profiles, and corrugated hot rolls. The company was incorporated in 1970 and is headquartered in Madrid, Spain.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on Spain Stock Exchange.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on

...

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0143421073', 'Innovative Solutions Ecosystem SA (ISE)', 'Innovative Solutions Ecosystem SA (ISE)', 'Service Point Solutions SA is a Spain-based company primarily engaged in the provision of commercial printing services. The Company is structured into four business lines: Digital Reprographics, which operates approximately 100 service centers specializing in printing and reproducing documents digitally; Document Management, which is mainly engaged in archiving digital documents and offering computer applications aimed to optimize operational costs related to the document flow; Postage, which provides distribution and mail delivery services, and Mailroom, which offers outsourcing services of mail-related activities. The Company is a parent of Grupo Service Point. The main shareholder of the Company is Paragon.', 'Innovative Solutions Ecosystem SA (ISE)(English: Innovative Solutions Eco

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105089009', 'Lleidanetworks Serveis Telematics SA (LLN)', 'Lleidanetworks Serveis Telematics SA (LLN)', 'Lleida Networks Serveis is a Spain-based company principally engaged in the telecommunications services. The Company offers short message services (SMS) and multimedia message services, such as e-Mails. The Company provides such solutions as registered electronic contracting, data validation, electronic registered communications, services for telecommunication operators, unregistered electronic communications. The Company offers its services to other companies, as well as councils and public administrations.', 'Lleidanetworks Serveis Telematics SA (LLN)(English: Lleidanetworks Serveis Telematics SA (LLN)) is a public company that is listed on Spain Stock Exchange.', 'Lleidanetworks Serveis Telematics SA (LLN)(English: Lleidanetworks Serveis Telematics SA (LLN)) is a public compan

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105223004', 'Gestamp Automocion SA (GEST)', 'Gestamp Automocion SA (GEST)', 'Gestamp Automoción, S.A. designs, develops, manufactures, and sells metal automotive components in Europe, North America, South America, and Asia. The company offers Body-in-White products, such as bonnets, roofs, doors, and mudguards, as well as other surface and assembly parts; and structural and crash-related elements that include floors, pillars, rails, and wheel arches. It also provides chassis parts, such as systems; frames and related parts comprising front and rear axles and couplings; control arms; and integrated couplings. In addition, the company offers mechanism parts, including hinges, door checks, electrical systems, and powered systems, as well as driver controls. It provides its products to manufacturers of light vehicles. The company was incorporated in 1997 and is headquartered in Madrid, 

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105344016', 'Tier1 Technology S.A. (TR1)', 'Tier1 Technology S.A. (TR1)', 'TIER 1 Technology, S.A. develops and implements software solutions. The company offers ERP attractor software for business management; Engage software for the management and coordination of suppliers; comerzzia, a commerce software for retail management; Lustrum asset management software; and SecurInvoice, a software for creation and management of electronic invoice. It also provides infrastructure services, such as datacenter, cloud computing, monitoring, communications, security, mobility, technological consulting, data center construction, supply and asset management, queue management, and audiovisual services; ICT support, maintenance, and outsourcing services; and consulting and business process outsourcing services. The company was founded in 2002 and is headquartered in Camas, Spain.', 'Tier1 Technolog

pass
pass
pass
fail
pass
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105091005', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)(English: Corpfin Capital Prime Retail II Socimi SA (YPR2)) is a public company that is listed on Spain Stock Exchange.', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)(English: Corpfin Capital Prime Retail II Socimi SA (YPR2)) is a public company that is listed on Spain Stock Exchange.', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)(English: Corpfin Capital Prime Retail II Socimi SA (YPR2)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'Corpfin Capital Prime Retail II Socimi SA (YPR2)(English: Corpfin Capital Prime Retail II Socimi SA (YPR2))

In [10]:
cover = open('Spain_2022_investing_2022-07-04_Chris-3.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [11]:
failList3 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/spain')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(2)
    
for sub in failList2:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ESP") 
        info.append("Espana") 
        info.append("Spain") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2200000000000 USD") 
        info.append("47450795") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ESPHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Spain, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain, Europe"
                post = "N/A"
            except:
                address1 = "Spain"
                address2 = "Spain, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Espanol")
        info.append("Spanish")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BME")
        info.append("Bolsas de Valores")
        info.append("Bolsas de Valores")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Spain")
        info.append("Spain, Europe")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList3.append(sub)
            
print(failList3)

C:\Users\user\AppData\Local\Temp\ipykernel_7632\2463955724.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0150480111', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA is a Spain-based company primarily engaged in the real estate sector. The Company’s activities include the acquisition and development of land, as well as the promotion and construction of non-residential properties, such as hotels, office buildings, shopping centers, senior residences, as well as logistics and industrial properties. The Company owns such companies as Gestora Inmobiliaria Besos SA, Edutaimet Sant Adria de Besos SA, Promociones Industriales y Financieras SA, Constructora Inbesos SA, Inbesos Sur SA and Nalcar 2000 SL, among others. On December 5, 2013, Commercial Court No. 1 of Zaragoza completed the common phase of insolvency proceedings. In December 2013, the trustee in bankruptcy presented final report of the Company required under artic

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0124244E34', 'Mapfre (MAP)', 'Mapfre (MAP)', 'Mapfre, S.A., engages in the insurance and reinsurance activities worldwide. It offers life, health, accident, savings and investment, retirement, burial, and travel and leisure insurance; and homeowner’s, automobile, third-party liability, family, and other insurance. The company also provides vehicle, third-party liability and asset, agriculture and livestock, commercial establishment, and other insurance products. In addition, it offers engineering and building, hull and aviation, transportation of goods, surety and credit, life and retirement, and other insurances and reinsurance products. The company offers its services to individuals, professionals, entrepreneurs, self-employed people, small and medium-sized enterprises, and large corporations. It distributes its products through a network of 4,942 direct and delegates; and 10,412 b

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0180907000', 'Unicaja Banco SA (UNI)', 'Unicaja Banco SA (UNI)', 'Unicaja Banco, S.A. provides various banking products and services to individuals and companies in Spain and internationally. It offers accounts, payments, and debit and credit cards; mortgages and personal loans; deposit products, stock exchange, pension plans, investment funds and portfolios, and savings insurance policies; and life, home, car, accident, health, and agricultural insurance, as well as SME and retail damage, and corporate liability insurance products. The company also provides cash management, short term and long-term financing, and investment services, as well as remote and mobile banking services. In addition, it engages in the property development and renewable energies activities; and invests in assets, securities, and financial companies. As of December 31, 2021, it had a network of 1,368 branches

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0132105018', 'Acerinox (ACX)', 'Acerinox (ACX)', 'Acerinox, S.A., through its subsidiaries, manufactures, transforms, and markets stainless steel products in Spain, the Americas, Africa, Asia, Oceania, and Europe. The company offers flat products, including coil cold rollings, hot rolled and black coils, teardrop steel or coils, and hot and cold rolled sheets, as well as roughing materials, discs, billets, and plates. It also provides long products, which include steel and color coated wires, corrugated wires, hexagonal wire rods, bars, hot and cold rebars, decorticated bars, black bars, steel profiles, and corrugated hot rolls. The company was incorporated in 1970 and is headquartered in Madrid, Spain.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on Spain Stock Exchange.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on

...

pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105273009', 'Grupo Ortiz Properties Socimi (YGOP)', 'Grupo Ortiz Properties Socimi (YGOP)', "Grupo Ortiz Properties SOCIMI SA is a Spain-based real estate company. The Company's main activity is the management of its diversified portfolio of assets, held for long-term rental purposes. The Company's portfolio consists of approximately 100.000 square meters of energy-efficient properties, both residential and business, located mainly in Madrid and including apartment buildings, offices, commercial premises, warehouses, parking structures and service stations. The Company divides its business into five segments, reflecting the type of properties: Offices, Residential, Warehouses, commercial premises and other, Parking and Service stations.", 'Grupo Ortiz Properties Socimi (YGOP)(English: Grupo Ortiz Properties Socimi (YGOP)) is a public company that is listed on Spain Stock Exchange.', 'Gru

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105225009', 'Robot (RBT)', 'Robot (RBT)', 'Robot SA is a Spain-based technology company. It designs, manufactures, installs and maintains systems for the automation and control of technical installations of buildings and energy savings, also known as Building Automation Systems (BAS). The Company focuses on all elements of the value chain with a focus on both the product and the service provided to its customers. It has carried out more than 600 installations in any type of building, including hotels, offices, shopping centers, universities and hospitals, among others, specializing in the hotel industry. The Company is active globally.', 'Robot (RBT)(English: Robot (RBT)) is a public company that is listed on Spain Stock Exchange.', 'Robot (RBT)(English: Robot (RBT)) is a public company that is listed on Spain Stock Exchange.', None, None, '',

fail
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105376000', 'Arima Real Estate SOCIMI SA (ARM)', 'Arima Real Estate SOCIMI SA (ARM)', 'Árima Real Estate is a Spanish company that listed on the Madrid Stock Exchange as a SOCIMI (Sociedad Anónima Cotizada de Inversión Inmobiliaria) and that is led by a fully-dedicated internal management team. It was created in 2018 with the aim of becoming the leading Spanish SOCIMI in the Madrid office market. The company is headed up by Luis María Arredondo (Chairman of the Board of Directors) and Luis López de Herrera-Oria (CEO) and also boasts the same highly-reputable management team that steered Axiare Patrimonio SOCIMI to success.', 'Arima Real Estate SOCIMI SA (ARM)(English: Arima Real Estate SOCIMI SA (ARM)) is a public company that is listed on Spain Stock Exchange.', 'Arima Real Estate SOCIMI SA (ARM)(English: Arima Real Estate SOCIMI SA (ARM)) is a public company that is list

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105548004', 'Grupo Ecoener SA (ECNER)', 'Grupo Ecoener SA (ECNER)', 'Grupo Ecoener, S.A.U., though its subsidiaries, engages in the renewable energy business in Spain and internationally. The company is involved in the development, engineering, procurement, construction, operation, and maintenance of hydropower facilities, wind farms, and solar photovoltaic plants, as well as power commercialization business. It has an installed capacity of 141 MW, including 52 MW of hydropower facilities; 73 MW of wind farms; and 16 MW of solar photovoltaic plants. The company was founded in 1988 and is headquartered in La Coruña, Spain. Grupo Ecoener, S.A.U. is a subsidiary of Ecoener, S.L.U.', 'Grupo Ecoener SA (ECNER)(English: Grupo Ecoener SA (ECNER)) is a public company that is listed on Spain Stock Exchange.', 'Grupo Ecoener SA (ECNER)(English: Grupo Ecoener SA (ECNER)) is a public company th

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105030003', 'Mercal Inmuebles SOCIMI SA (YMEI)', 'Mercal Inmuebles SOCIMI SA (YMEI)', 'Mercal Inmuebles Socimi, S.A. engages in the leasing and managing real estate properties. Its portfolio consists of offices, hospitals, land, and garages. The company is based in Madrid, Spain.', 'Mercal Inmuebles SOCIMI SA (YMEI)(English: Mercal Inmuebles SOCIMI SA (YMEI)) is a public company that is listed on Spain Stock Exchange.', 'Mercal Inmuebles SOCIMI SA (YMEI)(English: Mercal Inmuebles SOCIMI SA (YMEI)) is a public company that is listed on Spain Stock Exchange.', None, None, '', '', '', '630 17 95 63', '-', '', 'www.mercalinmuebles.es', '', '', '', '', '', 'Calle Orense 81 7th Floor, Spain', 'Calle Orense 81 7th Floor, Spain', '28020', '28020', '28020', '1', '', '', 'Listed', 'José Enrique García Ramos', '', 'CEO, Secretary & Director', 'CEO, Secretary & Director', 'Executive Board', '63

In [12]:
cover = open('Spain_2022_investing_2022-07-04_Chris-4.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [13]:
failList4 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/spain')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(2)
    
for sub in failList3:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ESP") 
        info.append("Espana") 
        info.append("Spain") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2200000000000 USD") 
        info.append("47450795") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ESPHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Spain, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain, Europe"
                post = "N/A"
            except:
                address1 = "Spain"
                address2 = "Spain, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Espanol")
        info.append("Spanish")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BME")
        info.append("Bolsas de Valores")
        info.append("Bolsas de Valores")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Spain")
        info.append("Spain, Europe")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList4.append(sub)
            
print(failList4)

C:\Users\user\AppData\Local\Temp\ipykernel_7632\3918871102.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0150480111', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA is a Spain-based company primarily engaged in the real estate sector. The Company’s activities include the acquisition and development of land, as well as the promotion and construction of non-residential properties, such as hotels, office buildings, shopping centers, senior residences, as well as logistics and industrial properties. The Company owns such companies as Gestora Inmobiliaria Besos SA, Edutaimet Sant Adria de Besos SA, Promociones Industriales y Financieras SA, Constructora Inbesos SA, Inbesos Sur SA and Nalcar 2000 SL, among others. On December 5, 2013, Commercial Court No. 1 of Zaragoza completed the common phase of insolvency proceedings. In December 2013, the trustee in bankruptcy presented final report of the Company required under artic

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0124244E34', 'Mapfre (MAP)', 'Mapfre (MAP)', 'Mapfre, S.A., engages in the insurance and reinsurance activities worldwide. It offers life, health, accident, savings and investment, retirement, burial, and travel and leisure insurance; and homeowner’s, automobile, third-party liability, family, and other insurance. The company also provides vehicle, third-party liability and asset, agriculture and livestock, commercial establishment, and other insurance products. In addition, it offers engineering and building, hull and aviation, transportation of goods, surety and credit, life and retirement, and other insurances and reinsurance products. The company offers its services to individuals, professionals, entrepreneurs, self-employed people, small and medium-sized enterprises, and large corporations. It distributes its products through a network of 4,942 direct and delegates; and 10,412 b

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0180907000', 'Unicaja Banco SA (UNI)', 'Unicaja Banco SA (UNI)', 'Unicaja Banco, S.A. provides various banking products and services to individuals and companies in Spain and internationally. It offers accounts, payments, and debit and credit cards; mortgages and personal loans; deposit products, stock exchange, pension plans, investment funds and portfolios, and savings insurance policies; and life, home, car, accident, health, and agricultural insurance, as well as SME and retail damage, and corporate liability insurance products. The company also provides cash management, short term and long-term financing, and investment services, as well as remote and mobile banking services. In addition, it engages in the property development and renewable energies activities; and invests in assets, securities, and financial companies. As of December 31, 2021, it had a network of 1,368 branches

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0178165017', 'Tecnicas Reunidas (TRE)', 'Tecnicas Reunidas (TRE)', 'Técnicas Reunidas, S.A., an engineering and construction company, engages in the design and management of industrial plant projects worldwide. It operates through Oil and Gas, Power, and Other Industries segments. The Oil and Gas segment offers engineering, procurement, and construction services in oil processing operations and chemical production and processing operations; services related to the natural gas production and extraction value chain, such as production, processing, storage, and transportation; constructs, revamps, and expands refining plants; designs and builds auxiliary services and other refining units; and designs and constructs monomers, polymers and plastics, chemical, and fertilizer producing and processing plants. The Power segment provides consulting, engineering, procurement, and construction s

...

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105270005', 'Netex Knowledge Factory SL (NTX)', 'Netex Knowledge Factory SL (NTX)', 'Netex Knowledge Factory S.A. develops applications and service-based-solutions to deliver digital learning in companies and academic institutions. The company’s portfolio includes learningCloud, a personalized learning environment that facilitates video, gamification, online/offline mobile learning, micro learning, spaced learning, social learning; contentCloud, a content production and management of digital content in one place; talentCloud, to develop reskilling and upskilling of organization. Netex Knowledge Factory S.A. was incorporated in 1997 and is headquartered in La Coruña, Spain.', 'Netex Knowledge Factory SL (NTX)(English: Netex Knowledge Factory SL (NTX)) is a public company that is listed on Spain Stock Exchange.', 'Netex Knowledge Factory SL (NTX)(English: Netex Knowledge Factory SL (N

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105357000', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated 

fail
pass
pass
pass
pass
fail
pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0171613005', 'Iffe Futura SA (IFFE)', 'Iffe Futura SA (IFFE)', 'Iffe Futura SA (IFFE)(English: Iffe Futura SA (IFFE)) is a public company that is listed on Spain Stock Exchange.', 'Iffe Futura SA (IFFE)(English: Iffe Futura SA (IFFE)) is a public company that is listed on Spain Stock Exchange.', 'Iffe Futura SA (IFFE)(English: Iffe Futura SA (IFFE)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'Iffe Futura SA (IFFE)(English: Iffe Futura SA (IFFE)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Spain', 'Spain', 'Spain, Europe', 'Spain, Europe', 'N/A', '15', '', '', 'Listed', 'Don Jesús Manuel Carne

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105425005', 'Plasticos Compuestos SA (KOM)', 'Plasticos Compuestos SA (KOM)', 'Plásticos Compuestos, S.A. engages in the design, production, and marketing of mineral filler concentrates, and color concentrates and additives. The company offers minerals concentrates for blown film, blow and injection moulding, raffia and ropes, thermoforming and sheet extrusion, pipes and profiles, and recycling industries under exfill; compostable resins under biokomp brand name; biodegradble in water product under Okean brand name; color masterbatches under kroma brand name; and black and white masterbatch. The company is headquartered in Barcelona, Spain.', 'Plasticos Compuestos SA (KOM)(English: Plasticos Compuestos SA (KOM)) is a public company that is listed on Spain Stock Exchange.', 'Plasticos Compuestos SA (KOM)(English: Plasticos Compuestos SA (KOM)) is a public company that is listed on Sp

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105432001', 'La Finca Global Assets Socimi SA (YLFG)', 'La Finca Global Assets Socimi SA (YLFG)', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', '', '', '

In [14]:
cover = open('Spain_2022_investing_2022-07-04_Chris-5.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [16]:
failList5 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/spain')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(2)
    
for sub in failList4:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ESP") 
        info.append("Espana") 
        info.append("Spain") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2200000000000 USD") 
        info.append("47450795") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ESPHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Spain, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Spain, Europe"
                post = "N/A"
            except:
                address1 = "Spain"
                address2 = "Spain, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Espanol")
        info.append("Spanish")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BME")
        info.append("Bolsas de Valores")
        info.append("Bolsas de Valores")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Spain")
        info.append("Spain, Europe")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList5.append(sub)
            
print(failList5)

C:\Users\user\AppData\Local\Temp\ipykernel_7632\2924935149.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0150480111', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA (IBES)', 'Nyesa Valores Corporacion SA is a Spain-based company primarily engaged in the real estate sector. The Company’s activities include the acquisition and development of land, as well as the promotion and construction of non-residential properties, such as hotels, office buildings, shopping centers, senior residences, as well as logistics and industrial properties. The Company owns such companies as Gestora Inmobiliaria Besos SA, Edutaimet Sant Adria de Besos SA, Promociones Industriales y Financieras SA, Constructora Inbesos SA, Inbesos Sur SA and Nalcar 2000 SL, among others. On December 5, 2013, Commercial Court No. 1 of Zaragoza completed the common phase of insolvency proceedings. In December 2013, the trustee in bankruptcy presented final report of the Company required under article 96

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0124244E34', 'Mapfre (MAP)', 'Mapfre (MAP)', 'Mapfre, S.A., engages in the insurance and reinsurance activities worldwide. It offers life, health, accident, savings and investment, retirement, burial, and travel and leisure insurance; and homeowner’s, automobile, third-party liability, family, and other insurance. The company also provides vehicle, third-party liability and asset, agriculture and livestock, commercial establishment, and other insurance products. In addition, it offers engineering and building, hull and aviation, transportation of goods, surety and credit, life and retirement, and other insurances and reinsurance products. The company offers its services to individuals, professionals, entrepreneurs, self-employed people, small and medium-sized enterprises, and large corporations. It distributes its products through a network of 4,942 direct and delegates; and 10,412 b

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0180907000', 'Unicaja Banco SA (UNI)', 'Unicaja Banco SA (UNI)', 'Unicaja Banco, S.A. provides various banking products and services to individuals and companies in Spain and internationally. It offers accounts, payments, and debit and credit cards; mortgages and personal loans; deposit products, stock exchange, pension plans, investment funds and portfolios, and savings insurance policies; and life, home, car, accident, health, and agricultural insurance, as well as SME and retail damage, and corporate liability insurance products. The company also provides cash management, short term and long-term financing, and investment services, as well as remote and mobile banking services. In addition, it engages in the property development and renewable energies activities; and invests in assets, securities, and financial companies. As of December 31, 2021, it had a network of 1,368 branches

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0132105018', 'Acerinox (ACX)', 'Acerinox (ACX)', 'Acerinox, S.A., through its subsidiaries, manufactures, transforms, and markets stainless steel products in Spain, the Americas, Africa, Asia, Oceania, and Europe. The company offers flat products, including coil cold rollings, hot rolled and black coils, teardrop steel or coils, and hot and cold rolled sheets, as well as roughing materials, discs, billets, and plates. It also provides long products, which include steel and color coated wires, corrugated wires, hexagonal wire rods, bars, hot and cold rebars, decorticated bars, black bars, steel profiles, and corrugated hot rolls. The company was incorporated in 1970 and is headquartered in Madrid, Spain.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on Spain Stock Exchange.', 'Acerinox (ACX)(English: Acerinox (ACX)) is a public company that is listed on

...

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105270005', 'Netex Knowledge Factory SL (NTX)', 'Netex Knowledge Factory SL (NTX)', 'Netex Knowledge Factory S.A. develops applications and service-based-solutions to deliver digital learning in companies and academic institutions. The company’s portfolio includes learningCloud, a personalized learning environment that facilitates video, gamification, online/offline mobile learning, micro learning, spaced learning, social learning; contentCloud, a content production and management of digital content in one place; talentCloud, to develop reskilling and upskilling of organization. Netex Knowledge Factory S.A. was incorporated in 1997 and is headquartered in La Coruña, Spain.', 'Netex Knowledge Factory SL (NTX)(English: Netex Knowledge Factory SL (NTX)) is a public company that is listed on Spain Stock Exchange.', 'Netex Knowledge Factory SL (NTX)(English: Netex Knowledge Factory SL (N

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105357000', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'Quid Pro Quo Alquiler Seguro Socimi (YQPQ)(English: Quid Pro Quo Alquiler Seguro Socimi (YQPQ)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated 

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105391009', 'Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)', 'Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)', 'Inmobiliaria Park Rose Iberoamericana SOCIMI, S.A. engages in leasing of commercial properties and office buildings in Spain and the United States. The company is based in Barcelona, Spain.', 'Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)(English: Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)) is a public company that is listed on Spain Stock Exchange.', 'Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)(English: Inmobiliaria Park Rose Iberoamericana SOCIMI SA (YPARK)) is a public company that is listed on Spain Stock Exchange.', None, None, '', '', '', '34 93 272 29 68', '-', '', 'www.parkroseinmobiliaria.com', '', '', '', '', '', 'C/Pau Claris, 162 5º 2ª, Spain', 'C/Pau Claris, 162 5º 2ª, Spain', '08037', '08037', '08037', '1'

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105425005', 'Plasticos Compuestos SA (KOM)', 'Plasticos Compuestos SA (KOM)', 'Plásticos Compuestos, S.A. engages in the design, production, and marketing of mineral filler concentrates, and color concentrates and additives. The company offers minerals concentrates for blown film, blow and injection moulding, raffia and ropes, thermoforming and sheet extrusion, pipes and profiles, and recycling industries under exfill; compostable resins under biokomp brand name; biodegradble in water product under Okean brand name; color masterbatches under kroma brand name; and black and white masterbatch. The company is headquartered in Barcelona, Spain.', 'Plasticos Compuestos SA (KOM)(English: Plasticos Compuestos SA (KOM)) is a public company that is listed on Spain Stock Exchange.', 'Plasticos Compuestos SA (KOM)(English: Plasticos Compuestos SA (KOM)) is a public company that is listed on Sp

pass
pass
pass
['ESP', 'Espana', 'Spain', 'UTC+01:00', '유럽', '2200000000000 USD', '47450795', '서유럽', 'ES0105432001', 'La Finca Global Assets Socimi SA (YLFG)', 'La Finca Global Assets Socimi SA (YLFG)', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', 'La Finca Global Assets Socimi SA (YLFG)(English: La Finca Global Assets Socimi SA (YLFG)) is a public company that is listed on Spain Stock Exchange. They are providing products/services in Spain, and they are operated as public company.', '', '', '

In [17]:
print(len(failList5))

130
